In [1]:
from cdep_db import get_con

In [3]:
import requests

In [4]:
from time import sleep

In [5]:
url = 'https://nominatim.openstreetmap.org/search?street={}&city=Dallas&county=Dallas&state=TX&country=USA&addressdetails=1&format=geojson'

In [6]:
update = "UPDATE cdep.address_lookup SET zip = %s, lat = %s, lon = %s WHERE address = %s;"

In [7]:
query = 'SELECT address FROM cdep.address_lookup WHERE queried IS False LIMIT 100;'

In [2]:
con = get_con()
cur = con.cursor()

In [8]:
cur.execute(query)

In [9]:
q = cur.fetchall()

In [18]:
for addr in q:
    r = requests.get(url.format(addr[0]))
    if r.json()['features']:
        zc = r.json()['features'][0]['properties']['address']['postcode'] if 'postcode' in r.json()['features'][0]['properties']['address'].keys() and r.json()['features'][0]['properties']['address']['postcode'].find(':') == -1 else None
        lon, lat = r.json()['features'][0]['geometry']['coordinates']
        cur.execute(update,(zc,lat,lon,addr[0]))
    cur.execute(f"UPDATE cdep.address_lookup SET queried = True WHERE address = '{addr[0]}';")
    con.commit()
    sleep(1)

In [83]:
con.rollback() ## for the transaction errors in the database

## figuring out the missing postcode issue

In [14]:
if r.json()['features'][0]['properties']['address']['postcode']:
    print('true')

KeyError: 'postcode'

In [15]:
type(r.json()['features'][0]['properties']['address'])

dict

In [17]:
if 'postcode' in r.json()['features'][0]['properties']['address'].keys():
    print('true')
else:
    print('false')

false


## Figuring out the combo ZIP

In [85]:
zc = r.json()['features'][0]['properties']['address']['postcode'] if r.json()['features'][0]['properties']['address']['postcode'].find(':') == -1 else None

In [70]:
r.json()['features'][0]['properties']['address']['postcode'].find(':') = -1

'75231:75238'

In [57]:
if r.json()['features']:
    print('true')

In [10]:
addr = q[0]

In [17]:
r = requests.get(url.format(addr[0]))

In [62]:
r.json()

{'type': 'FeatureCollection',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'features': [{'type': 'Feature',
   'properties': {'place_id': 188304218,
    'osm_type': 'way',
    'osm_id': 468104999,
    'display_name': '10062, Royal Lane, Lake Highlands, Dallas, Dallas County, Texas, 75231:75238, United States of America',
    'place_rank': 30,
    'category': 'building',
    'type': 'apartments',
    'importance': 0.431,
    'address': {'house_number': '10062',
     'road': 'Royal Lane',
     'quarter': 'Lake Highlands',
     'city': 'Dallas',
     'county': 'Dallas County',
     'state': 'Texas',
     'postcode': '75231:75238',
     'country': 'United States of America',
     'country_code': 'us'}},
   'bbox': [-96.714778, 32.89388, -96.7144269, 32.8939651],
   'geometry': {'type': 'Point', 'coordinates': [-96.71460245, 32.89392255]}},
  {'type': 'Feature',
   'properties': {'place_id': 190306420,
    'osm_type': 'way',
    'osm_id': 468103186,

In [ ]:
## Initial parsing of JSON response for UPDATE stateme

In [26]:
zc = r.json()['features'][0]['properties']['address']['postcode']

In [33]:
lon, lat = r.json()['features'][0]['geometry']['coordinates']

In [47]:
cur.execute(update.format(zc,lat,lon,addr[0]))

In [48]:
con.commit()

In [46]:
con.rollback()

In [39]:
cur.execute('SELECT * FROM cdep.address_lookup LIMIT 10;')
q = cur.fetchall()

In [40]:
q

[(1,
  None,
  '10000 BRUTON RD',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (2,
  None,
  '10000 CHECOTA DR',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (3,
  None,
  '10000 EDGECOVE DR',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (4,
  None,
  '10000 HARRY HINES BLVD',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (5,
  None,
  '10000 LAKE JUNE RD',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (6,
  None,
  '10000 L B J FWY',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (7,
  None,
  '10000 MARSH LN',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (8,
  None,
  '10000 MESITA DR',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (9,
  None,
  '10000 N CENTRAL EXPY',
  None,
  None,
  None,
  Non

In [41]:
cur.description

(Column(name='id', type_code=23),
 Column(name='id_correct', type_code=23),
 Column(name='address', type_code=25),
 Column(name='zip', type_code=1043),
 Column(name='lat', type_code=1700),
 Column(name='lon', type_code=1700),
 Column(name='x', type_code=1700),
 Column(name='y', type_code=1700),
 Column(name='beat', type_code=1700),
 Column(name='ra', type_code=1700),
 Column(name='division', type_code=25),
 Column(name='sector', type_code=1700),
 Column(name='council_dist', type_code=1043))

In [ ]:
cur.execute